---
title: Genequery Logic and tests.
authors:
- Rohan
tags:
- Fisher Exact test + code
- Fisher Enrichment Analysis test + code
- overlap 
- process .gmt file 

created_at: 2019-04-29
updated_at: 2019-05-12
tldr: Decoding Genequery logic and getting results 100%.
---

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import math
import re

### Reading query file and preparing list

In [17]:
col = ["ids"]

hypoxia_sign = pd.read_csv("query_1(gQ_ex).txt", header = None, names = col)

QUERY = hypoxia_sign['ids'].values

### Computing pValue using Fisher Exact Test

In [9]:
universeSize = 7000     
logFactorials = np.repeat( 0.0 , universeSize+1 )

def make_logF(logFactorials):
    for i in range(1,universeSize):
        logFactorials[i] = logFactorials[i-1] + math.log(i)
    return(logFactorials)
        
    
logFactorials = make_logF(logFactorials)

def calculateHypergeomP(a,b,c,d):
    return(np.exp(logFactorials[a+b]+logFactorials[c+d]+logFactorials[a+c]+logFactorials[b+d]-logFactorials[a+b+c+d]-logFactorials[a]-logFactorials[b]-logFactorials[c]-logFactorials[d]))

def righttailPvalue(a,b,c,d):
    ra = a
    rb = b
    rc = c
    rd = d
    
    if(ra+rb+rc+rd < len(logFactorials)):
        print("Sum of arguments within limit")
    else:
        print("Sum of the arguments must be not greater than universe: $a + $b + $c + $d > ${logFactorials.size - 1}")
        return (None)
        
        
    pSum = 0.0
    p = calculateHypergeomP(ra,rb,rc,rd)
    
    while((rc >= 0) and (rb >= 0)):
        if(p != 0):
            pSum += p
            
        if((rb==0) or (rc == 0)):
            
            break
        
        ra = ra+1
        rb = rb-1
        rc = rc-1
        rd = rd+1
    

        p= calculateHypergeomP(ra,rb,rc,rd)
          
    return(pSum)

### Testing fisherExact Test

In [10]:
righttailPvalue(16,31,11,6800)

Sum of arguments within limit


1.6629173637850388e-29

### Convert strings and process
### - processing modules and convert to numeric for Overlap

In [24]:
def convert(module):
    module = module.rstrip()
    
    mod = module.split(',')
    mod = list(map(int, mod))
    
    return(mod)

### Process GSE as file object to DataFrame

In [23]:
def process_gse(open_var):
    
    DF = pd.DataFrame(open_var)
    DF.columns = ["mod_ID"]
    
    DF['mod_number'] = DF['mod_ID'].str.split('#').str[-1]
    
    DF['entrez'] = DF['mod_number'].str.split('\t').str[-1]
    
    DF['mod_ID'] = DF['mod_ID'].str.split('#').str[0]
    
    DF['mod_number'] = DF['mod_number'].str.split('\t').str[0]

    DF["Entrez"] = DF.entrez.apply(convert)
    del DF["entrez"]
    
    return(DF)
    

### Compute Overlap
### - returns intersection Size 

In [11]:
def Overlap(module):
    
    ovlp = set(QUERY).intersection(set(module))
    
    return(len(ovlp))

### Compute Bonferroni Significance Value as Adjusted pValue

### - matrix here is a single GSE_ID as a object (Dataframe)
### - m_count is the moduleCount for the species

In [21]:
min_logp = -325.0
bonferroniMaxPvalue = 0.01

def bonfS(m_count, matrix):
    
    mod_count = m_count
    GSE_obj = matrix
    G_count = GSE_obj.Size.count()
    
    
    queryUniverseOverlap = sum(GSE_obj.Intersection_size) 
    
    if(queryUniverseOverlap == 0):
        return(None)

    universeSize = sum(GSE_obj.Size) 
    
    
    
    GSE_obj["pval"] = 0.0
    GSE_obj["apval"] = 0.0
    GSE_obj["logpval"] = 0.0
    GSE_obj["logapval"] = 0.0
    
   
   # m = 0
    
    
    for i in GSE_obj.index:
        if(GSE_obj.at[i,'Intersection_size'] > 0):
        
            moduleAndQuery = GSE_obj.at[i,'Intersection_size']
        
            moduleAndNotQuery = GSE_obj.at[i,'Size'] - GSE_obj.at[i,'Intersection_size']
        
            queryAndNotModule = queryUniverseOverlap - moduleAndQuery
        
            restOfGenes = universeSize - moduleAndQuery - queryAndNotModule - moduleAndNotQuery
        
        
            pvalue = righttailPvalue(moduleAndQuery,moduleAndNotQuery,queryAndNotModule,restOfGenes)
        
        
            GSE_obj.at[i,"pval"] = pvalue
        
        # Computing AdjustedpValue
        
            adjustedPvalue = pvalue * mod_count
        
        
            if (adjustedPvalue <= bonferroniMaxPvalue):
                GSE_obj.at[i,"apval"] = adjustedPvalue
                
                
                if(pvalue > 0):
                    GSE_obj.at[i,"logpval"] = math.log10(pvalue)
                else:
                    GSE_obj.at[i,"logpval"] = min_logp
                
                
                if(adjustedPvalue > 0):
                    GSE_obj.at[i,"logapval"] = math.log10(adjustedPvalue)
                else:
                    GSE_obj.at[i,"logapval"] = min_logp

                    
            else:
                GSE_obj.at[i,"apval"] = None
        
       # m = i
    
    
    #print(m)    
    
    #df.isnull().values.any()
    
    #nl = GSE_obj[GSE_obj.apval.isnull()]
    #print(n)
    #c = nl.Size.count()
    
    #nrow = GSE_obj[GSE_obj.apval.isnull()]
    #print(nan_rows)
    
    #if(c == G_count):
    #    return(None)
    
    #if(c != 0):
    #    GSE_obj = GSE_obj[GSE_obj.apval.notnull()]
    
    #print(len(GSE_obj[GSE_obj.apval.isnull()]))
    
    #if(GSE_obj.pval)
    
    #GSE_obj["logpval"] = [math.log10(x) for x in GSE_obj.pval]
    
    #GSE_obj["logapval"] = [math.log10(x) for x in GSE_obj.apval]
    
    
    #print(df)
    
    return(GSE_obj)
        

### Compute Results of Genequery and return ranked list

### - rank_func takes Signature list and species.gmt as input

In [12]:

def rank_func(query, Species_file):
    
    spm = Species_file
    
    fil = open(spm, "r")
    sp_list = fil.readlines()
    
    moduleCount = len(sp_list)        
    print(moduleCount)
    
    df = process_gse(sp_list)
    fil.close()
    
    df["Size"] = [len(x) for x in df.Entrez]
    
    df["Intersection_size"] = df.Entrez.apply(Overlap)
    
    
    g_list = list(df.groupby("mod_ID"))
    
    array = []
    
    for i in range(0,len(g_list)):
        lst_1 = g_list[i]
        lst_2 = lst_1[1]
        if(lst_2["Intersection_size"].sum() == 0):
            continue
        else:
            df_obj = lst_2
            enrch_modules = bonfS(moduleCount, df_obj)    
        
        
        array.append(enrch_modules)
    
    
    #print(array)
    
    
    return(array)

### Reading the Module and parsing to rank function

### - " result " array stores the results for gea computation

In [13]:

FILE = "mm_modules.txt"

## Results of computation are stored as array in " result "

# result = rank_func(QUERY, FILE)

### Concatenate results as one table

In [4]:
# fdf = pd.concat(result)

In [37]:
RESULT = fdf[ fdf["logapval"] < 0 ]

In [39]:
gQ_res = RESULT.sort_values(['logapval'], ascending=[True])

In [14]:
# gQ_res

### Getting series column

In [3]:
# gQ_res['series'] = gQ_res['mod_ID'].str.split('_').str[0]


In [70]:
gQ_res = gQ_res[gQ_res["mod_number"] != '0' ]

In [16]:
# gQ_res

### Reading Original Genequery results

In [47]:
objt = pd.read_csv("genequery_search_result_16-50_4-5-2019.csv")

In [17]:
# objt

In [74]:
gQ_res.to_csv("results_gQ.csv", sep='\t', encoding='utf-8')


### Total number of modules in results of genequery

In [72]:
gQ_res.count()

mod_ID               415
mod_number           415
Entrez               415
Size                 415
Intersection_size    415
pval                 415
apval                415
logpval              415
logapval             415
series               415
dtype: int64